In [1]:
#import dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func, inspect

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base

# from sqlalchemy import create_engine
# from sqlalchemy import inspect

In [2]:
# Step 1: Read CSV Files into DataFrames
industry_groups_df = pd.read_csv('Resources/05_Industry_Groups.csv')
top_ten_df = pd.read_csv('Resources/04_top_ten_with_industryID.csv')
top_ten_historic_df = pd.read_csv('Resources/03_top_ten_historic_5y.csv')
fundamental_df_clean = pd.read_csv('Resources/07_ASX_Fundamental_Final_Clean.csv')
industry_groups_df.head()

,industry_name,industry_id
0,Automobiles & Components,1
1,Banks,2
2,Capital Goods,3
3,Class Pend,4
4,Commercial & Professional Services,5


In [3]:
fundamental_df_clean.head(12)

,fundamental_ID,Ticker,company_name,industry_gp,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,...,sharesIssued,Sector,similarCompanies,EPS,DPS,bookValuePerShare,Breakdown,Recommendation,lastUpdated,PE
0,1,ARB,ARB CORPORATION LIMITED.,Automobiles & Components,31.260,1.02 (3.37%),$29.88 - $31.35,160245.0,245483.0,31.070,...,8.222044e+07,Consumer Cyclical,CTD / WEB / SUL,1.079,0.6200,0.600,6 Buy · 5 Hold · 2 Sell,Hold,01/10/23,28.971
1,2,GUD,G.U.D. HOLDINGS LIMITED,Automobiles & Components,11.080,0.17 (1.56%),$10.77 - $11.35,366353.0,441797.0,10.950,...,1.408947e+08,Consumer Cyclical,SGR / LNW / SKC,0.693,0.3900,0.526,11 Buy · 2 Hold · 0 Sell,Strong Buy,01/10/23,15.988
2,3,PWH,PWR HOLDINGS LIMITED,Automobiles & Components,10.120,0.07 (0.70%),$9.96 - $10.92,37398.0,127350.0,10.160,...,1.004841e+08,Consumer Cyclical,BGP / CTT / AX1,0.217,0.1250,0.722,7 Buy · 4 Hold · 1 Sell,Buy,01/10/23,46.636
3,4,SFC,SCHAFFER CORPORATION LIMITED,Automobiles & Components,16.100,0.65 (-3.88%),$16.15 - $17.10,5570.0,758.0,16.750,...,1.356956e+07,Consumer Cyclical,NDO / ALG / BST,1.811,0.9000,3.324,NaN,NaN,NaN,8.890
4,5,AHL,ADRAD HOLDINGS LIMITED,Automobiles & Components,0.890,0.00 (0.00),$0.845 - $0.90,0.0,11146.0,NaN,...,8.128517e+07,Consumer Cyclical,ISU / KED / CCX,0.084,0.0233,0.986,NaN,NaN,NaN,10.595
5,6,VMT,VMOTO LIMITED,Automobiles & Components,0.155,0.005 (3.33%),$0.145 - $0.155,82262.0,432421.0,0.155,...,2.902403e+08,Consumer Cyclical,DNA / BBT / MFD,0.025,0.0000,0.202,NaN,NaN,NaN,6.200
6,7,RPM,RPM AUTOMOTIVE GROUP LIMITED,Automobiles & Components,0.081,0.0005 (0.62%),$0.078 - $0.085,92000.0,26007.0,0.081,...,1.954481e+08,Consumer Cyclical,XRG / ABV / CLU,0.010,0.0000,0.031,NaN,NaN,NaN,8.100
7,8,ABV,ADVANCED BRAKING TECHNOLOGY LTD,Automobiles & Components,0.044,0.002 (4.76%),$0.043 - $0.046,231792.0,154298.0,0.042,...,3.794669e+08,Consumer Cyclical,XRG / RPM / UCW,0.004,0.0000,0.017,NaN,NaN,NaN,11.000
8,9,DDT,DATADOT TECHNOLOGY LIMITED,Automobiles & Components,0.004,0.00 (0.00),$0.004 - $0.005,286645.0,1241239.0,0.004,...,1.210953e+09,Consumer Cyclical,MHI / SIX / AUA,0.000,0.0000,0.008,NaN,NaN,NaN,0.000
9,10,SIX,SPRINTEX LIMITED,Automobiles & Components,0.014,0.00 (0.00),$0.012 - $0.014,176.0,555355.0,0.014,...,3.389980e+08,Consumer Cyclical,AUA / MHI / DDT,-0.020,0.0000,0.003,NaN,NaN,NaN,-0.700


In [4]:
# #trying to see if i had any null values in 'Date' column of top_ten_historic_df
print(top_ten_historic_df['Ticker'].isnull().sum())

0


In [5]:
industry_groups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   industry_name  27 non-null     object
 1   industry_id    27 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 564.0+ bytes


In [6]:
fundamental_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fundamental_ID     259 non-null    int64  
 1   Ticker             259 non-null    object 
 2   company_name       259 non-null    object 
 3   industry_gp        259 non-null    object 
 4   lastPrice          259 non-null    float64
 5   Change             259 non-null    object 
 6   Bid_Ask            259 non-null    object 
 7   volumePerDay       259 non-null    float64
 8   volume4wAvg        259 non-null    float64
 9   Open               245 non-null    float64
 10  dayRange           259 non-null    object 
 11  prevClose          259 non-null    object 
 12  lastTrade          259 non-null    object 
 13  oneWeek            259 non-null    object 
 14  oneMonth           259 non-null    object 
 15  YTD2023            259 non-null    object 
 16  oneYear            259 non

In [7]:
#convert datatype to date
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date'])


In [8]:
top_ten_historic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60881 entries, 0 to 60880
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Ticker     60881 non-null  object        
 1   Date       60881 non-null  datetime64[ns]
 2   Open       60881 non-null  float64       
 3   High       60881 non-null  float64       
 4   Low        60881 non-null  float64       
 5   Close      60881 non-null  float64       
 6   Adj Close  60881 non-null  float64       
 7   Volume     60881 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 3.7+ MB


In [9]:
#top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date']).apply(lambda x: x.date())

In [10]:
#print(top_ten_historic_df['Date'].unique())

In [9]:
industry_groups_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   industry_name  27 non-null     object
 1   industry_id    27 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 564.0+ bytes


In [10]:
# Define Database Schema
Base = declarative_base()

class IndustryGroups(Base):
    __tablename__ = 'industry_groups'
    
    industry_name = Column(String)
    industry_id = Column(Integer, primary_key=True)
    

class TopTen(Base):
    __tablename__ = 'top_ten'
    
    ticker = Column(String, primary_key=True)
    company_name = Column(String)
    market_cap = Column(Float)
    industry_id = Column(Integer, ForeignKey('industry_groups.industry_id'))

class TopTenHistoric(Base):
    __tablename__ = 'top_ten_historic'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column('Adj Close', Float)
    Volume = Column(Integer)
    
class Fundamental(Base):
    __tablename__ = 'fundamental'
    
    id = Column (Integer, primary_key=True)
    Ticker = Column(String,ForeignKey('top_ten.ticker'))
    company_name = Column(String) 
    industry_gp = Column(String) 
    lastPrice = Column(Float)         
    Change = Column(String)       
    Bid_Ask = Column(String)           
    volumePerDay = Column(Float)         
    volume4wAvg = Column(Float)          
    Open = Column(Float)              
    dayRange = Column(String)              
    prevClose = Column(String)        
    lastTrade = Column(String)            
    oneWeek = Column(String)             
    oneMonth = Column(String)           
    YTD2023 = Column(String)            
    oneYear = Column(String)            
    vsSectorOneYr = Column(String)         
    vsASX200OneYr = Column(String)         
    marketCap = Column(Float)          
    ASXRank = Column(String)            
    sectorRank = Column(String)         
    sharesIssued = Column(Float)       
    Sector = Column(String)               
    similarCompanies = Column(String)     
    EPS = Column(Float)                 
    DPS = Column(Float)                
    bookValuePerShare = Column(Float)    
    Breakdown = Column(String)          
    Recommendation = Column(String)       
    lastUpdated = Column(String)                     
    PE= Column(Float) 
        
#Connect to the Database and Create Tables
# Replace 'sqlite:///asx.db' with your actual database connection string
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
Base.metadata.create_all(engine)


## Need to run this to put the data to the table and commit

In [11]:
from sqlalchemy.orm import sessionmaker

#Insert DataFrames into Database
Session = sessionmaker(bind=engine)


session = Session()

# Insert IndustryGroups
session.bulk_insert_mappings(IndustryGroups, industry_groups_df.to_dict(orient="records"))

# Insert TopTen
session.bulk_insert_mappings(TopTen, top_ten_df.to_dict(orient="records"))

# Insert TopTenHistoric
session.bulk_insert_mappings(TopTenHistoric, top_ten_historic_df.to_dict(orient="records"))

# Insert ASX_Fundamental
session.bulk_insert_mappings(Fundamental, fundamental_df_clean.to_dict(orient="records"))


session.commit()


### 1. Job to inspect the tables in the sqlite database

In [12]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [13]:
# Create an sqlalchemy engine to connect to the database
engine = create_engine('sqlite:///Resources/top_ten_asx_new.db')

In [14]:
# Create inspector object and use it to reflect the database and get a list of table names
inspector = inspect(engine)

# Get a list of table names in the database
table_names = inspector.get_table_names()

# Print the table names
for table_name in table_names:
    print(table_name)

fundamental
industry_groups
top_ten
top_ten_historic


### 2. Job to print all the columsn of each table in the database

In [15]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

# Create an sqlalchemy engine to connect to the database
engine = create_engine('sqlite:///Resources/top_ten_asx_new.db')

# Create an inspector object
inspector = inspect(engine)

# Get a list of table names in the database
table_names = inspector.get_table_names()

# Iterate through the table names and print the columns for each table
for table_name in table_names:
    print(f"Table: {table_name}")
    columns = inspector.get_columns(table_name)
    for column in columns:
        print(f"Column: {column['name']} - Type: {column['type']}")
    print("\n")

Table: fundamental
Column: id - Type: INTEGER
Column: Ticker - Type: VARCHAR
Column: industry_gp - Type: VARCHAR
Column: lastPrice - Type: FLOAT
Column: Change - Type: VARCHAR
Column: Bid_Ask - Type: VARCHAR
Column: volumePerDay - Type: FLOAT
Column: volume4wAvg - Type: FLOAT
Column: Open - Type: FLOAT
Column: dayRange - Type: VARCHAR
Column: prevClose - Type: VARCHAR
Column: lastTrade - Type: VARCHAR
Column: oneWeek - Type: VARCHAR
Column: oneMonth - Type: VARCHAR
Column: YTD2023 - Type: VARCHAR
Column: oneYear - Type: VARCHAR
Column: vsSectorOneYr - Type: VARCHAR
Column: vsASX200OneYr - Type: VARCHAR
Column: marketCap - Type: FLOAT
Column: ASXRank - Type: VARCHAR
Column: sectorRank - Type: VARCHAR
Column: sharesIssued - Type: FLOAT
Column: Sector - Type: VARCHAR
Column: similarCompanies - Type: VARCHAR
Column: EPS - Type: FLOAT
Column: DPS - Type: FLOAT
Column: bookValuePerShare - Type: FLOAT
Column: Breakdown - Type: VARCHAR
Column: Recommendation - Type: VARCHAR
Column: lastUpdated

### 3. Fetch the data from selected table

In [16]:
# SQLAlchemy
from pathlib import Path
from sqlalchemy import create_engine, text


# Create an sqlalchemy engine to connect to the database
engine = create_engine('sqlite:///Resources/top_ten_asx_new.db')
connection = engine.connect()

In [17]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM industry_groups"))

In [18]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM fundamental LIMIT 5"))

In [19]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM top_ten LIMIT 5"))

In [20]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM top_ten_historic LIMIT 5"))

In [21]:
# Once this is executed, we need to run the codes again to populate data
for record in data:
    print(record)

(1, 'ARB', '2018-11-05', 17.1299991607666, 17.540000915527344, 16.3700008392334, 17.149999618530273, None, 1106127)
(2, 'ARB', '2018-11-12', 17.059999465942383, 17.3700008392334, 16.655000686645508, 17.100000381469727, None, 1411720)
(3, 'ARB', '2018-11-19', 17.0, 17.700000762939453, 16.329999923706055, 17.1200008392334, None, 917302)
(4, 'ARB', '2018-11-26', 17.020000457763672, 17.559999465942383, 16.75, 16.8700008392334, None, 1240321)
(5, 'ARB', '2018-12-03', 17.3799991607666, 17.489999771118164, 16.110000610351562, 16.309999465942383, None, 1164860)
